In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

In [ ]:
squence_len=11
batch_size=1
input_dimension=512
d_model=512
x=torch.randn((batch_size,squence_len,input_dimension))
x

tensor([[[-0.7058, -0.7661, -1.1527,  ..., -0.4591,  0.5791, -0.3287],
         [ 0.0336, -1.2009,  0.5701,  ...,  1.0844,  0.7130, -0.3537],
         [-0.1329,  0.7946,  0.3595,  ...,  1.0061,  0.9566,  1.0486],
         ...,
         [-1.6452,  1.3200,  0.3463,  ...,  0.4655,  0.5920,  0.0310],
         [-1.0386, -0.4613, -1.8377,  ..., -0.2011, -0.0178, -0.7555],
         [ 0.5962, -1.1441, -0.2673,  ..., -1.4776, -0.3318,  0.4743]]])

ek batch mein 1 sentence jaa rha hai jis mein 11 words hai, each word have 512 dimension

In [ ]:
x.shape

torch.Size([1, 11, 512])

In [ ]:
qkv=nn.Linear(input_dimension,3*d_model)

3 layers with input dim and d_model dimension each

In [ ]:
qkv

Linear(in_features=512, out_features=1536, bias=True)

In [ ]:
qkv1=qkv(x)

In [ ]:
qkv1.shape

torch.Size([1, 11, 1536])

In [ ]:
num_head=8
head_dim=d_model//num_head

In [ ]:
head_dim

64

In [ ]:
qkv2=qkv1.reshape(batch_size,squence_len,num_head,3*head_dim)

In [ ]:
qkv2.shape

torch.Size([1, 11, 8, 192])

In [ ]:
qkv3=qkv2.permute(0,2,1,3)

In [ ]:
qkv3.shape

torch.Size([1, 8, 11, 192])

In [ ]:
q,k,v=qkv3.chunk(3,dim=-1)

In [ ]:
q.shape,k.shape,v.shape

(torch.Size([1, 8, 11, 64]),
 torch.Size([1, 8, 11, 64]),
 torch.Size([1, 8, 11, 64]))

dk=64 here

attention with multihead attention

In [ ]:
similarity_score=torch.matmul(q,k.transpose(-2,-1))

In [ ]:
similarity_score.shape

torch.Size([1, 8, 11, 11])

In [ ]:
k.size()

torch.Size([1, 8, 11, 64])

In [ ]:
dk=k.size()[-1]
dk

64

In [ ]:
scaled=similarity_score/math.sqrt(dk)

In [ ]:
scaled.shape

torch.Size([1, 8, 11, 11])

mask

In [ ]:
mask=torch.full(scaled.size(),float('-inf'))
mask

tensor([[[[-inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
          [-inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
          [-inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
          [-inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
          [-inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
          [-inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
          [-inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
          [-inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
          [-inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
          [-inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
          [-inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf]],

         [[-inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
          [-inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -in

In [ ]:
mask.shape

torch.Size([1, 8, 11, 11])

In [ ]:
mask=torch.triu(mask,diagonal=1)
mask

tensor([[[[0., -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
          [0., 0., -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
          [0., 0., 0., -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
          [0., 0., 0., 0., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
          [0., 0., 0., 0., 0., -inf, -inf, -inf, -inf, -inf, -inf],
          [0., 0., 0., 0., 0., 0., -inf, -inf, -inf, -inf, -inf],
          [0., 0., 0., 0., 0., 0., 0., -inf, -inf, -inf, -inf],
          [0., 0., 0., 0., 0., 0., 0., 0., -inf, -inf, -inf],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., -inf, -inf],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., -inf],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

         [[0., -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
          [0., 0., -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
          [0., 0., 0., -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
          [0., 0., 0., 0., -inf, -inf, -inf, -inf,

In [ ]:
mask[0][0]

tensor([[0., -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., 0., -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., -inf],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [ ]:
masked_score=scaled+mask

In [ ]:
masked_score[0][0]

tensor([[ 0.3905,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf,
            -inf,    -inf,    -inf],
        [-0.0498, -0.2837,    -inf,    -inf,    -inf,    -inf,    -inf,    -inf,
            -inf,    -inf,    -inf],
        [-0.6788,  0.0678, -0.2837,    -inf,    -inf,    -inf,    -inf,    -inf,
            -inf,    -inf,    -inf],
        [ 0.1294,  0.0779,  0.3292,  0.0499,    -inf,    -inf,    -inf,    -inf,
            -inf,    -inf,    -inf],
        [-0.3825, -0.0866,  0.0288, -0.5027,  0.2944,    -inf,    -inf,    -inf,
            -inf,    -inf,    -inf],
        [ 0.1757, -0.0935, -0.0236,  0.4888, -0.3184,  0.0528,    -inf,    -inf,
            -inf,    -inf,    -inf],
        [ 0.0853, -0.3604, -0.4833,  0.4891,  0.1285,  0.1154, -0.1151,    -inf,
            -inf,    -inf,    -inf],
        [ 0.2207, -0.1523, -0.1798, -0.1724,  0.2032,  0.3734, -0.1203, -0.3106,
            -inf,    -inf,    -inf],
        [-0.1574,  0.0821,  0.0254,  0.3346,  0.0742, -0

In [ ]:
attention=F.softmax(masked_score,dim=-1)

In [ ]:
attention

tensor([[[[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000],
          [0.5582, 0.4418, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000],
          [0.2177, 0.4592, 0.3231, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000],
          [0.2442, 0.2320, 0.2982, 0.2256, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000],
          [0.1491, 0.2004, 0.2249, 0.1322, 0.2934, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000],
          [0.1836, 0.1403, 0.1505, 0.2512, 0.1120, 0.1624, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000],
          [0.1516, 0.0971, 0.0858, 0.2270, 0.1583, 0.1562, 0.1241, 0.0000,
           0.0000, 0.0000, 0.0000],
          [0.1543, 0.1063, 0.1034, 0.1042, 0.1516, 0.1798, 0.1097, 0.0907,
           0.0000, 0.0000, 0.0000],
          [0.0972, 0.1236, 0.1167, 0.1590, 0.1226, 0.0877, 0.1172, 0.1183,
           0.0576, 0.0000, 0.0000],
 

In [ ]:
attention.shape

torch.Size([1, 8, 11, 11])

In [ ]:
contextual_embedding=torch.matmul(attention,v)

In [ ]:
contextual_embedding

tensor([[[[-3.2498e-01,  2.5610e-01, -3.9876e-01,  ...,  4.7484e-01,
            6.6170e-01, -6.2078e-03],
          [ 2.0282e-01,  1.4803e-01,  2.6623e-01,  ...,  6.4870e-02,
            5.5881e-01, -1.9879e-01],
          [ 4.2321e-01, -1.6296e-01,  3.1573e-01,  ..., -2.1202e-01,
            6.4225e-01, -1.8281e-01],
          ...,
          [-1.1761e-01,  2.2793e-02,  3.2186e-01,  ...,  1.4609e-01,
            3.0952e-01, -2.0116e-01],
          [-7.0185e-02, -4.1565e-02,  1.0201e-01,  ...,  4.4694e-02,
            2.1213e-01, -1.5755e-01],
          [-1.3517e-01,  5.6407e-02,  1.3712e-01,  ...,  8.6284e-02,
            2.1070e-01, -1.6137e-01]],

         [[-3.3217e-01, -7.4240e-01,  1.0724e+00,  ...,  7.3515e-01,
            4.1799e-01, -4.0547e-01],
          [-5.0294e-01, -5.2192e-01,  5.2951e-01,  ...,  8.1167e-01,
            1.1316e-01, -1.1998e-01],
          [-3.4313e-01, -3.0795e-01,  8.2566e-01,  ...,  5.9852e-01,
           -4.4560e-03, -2.4928e-01],
          ...,
     

In [ ]:
contextual_embedding.shape

torch.Size([1, 8, 11, 64])

In [ ]:
contextual_embedding.transpose(1,2).shape

torch.Size([1, 11, 8, 64])

output should be of same dim as input

In [ ]:
contextual_embedding.reshape(batch_size,squence_len,num_head*head_dim).shape

torch.Size([1, 11, 512])